In [99]:
%%bash 
git remote show origin


fatal: could not read Username for 'https://github.com': No such device or address


CalledProcessError: Command 'b'git remote show origin\n'' returned non-zero exit status 128.

In [16]:
%%bash
mkdir ML_training_01
chown -R jupyter:jupyter /home/jupyter/ML_training_01

In [15]:
import os
os.chdir('ML_training_01/')

In [98]:
%%bash
# Iniciar un repositorio
git init

Reinitialized existing Git repository in /home/jupyter/ML_training_01/.git/


In [100]:
%%bash
github_user='JAPJ182'
github_repo='ML_training_01'
 
 

In [101]:
%%bash 
# revisar que commits tenemos para cargar
git status

On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Course_1.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [102]:
%%bash
# Seleccionamos que cosas queremos cargar al repositorio
git add Course_1.ipynb

In [103]:
%%bash
#Se crea el comit dentro de nuestro repositorio local
git commit -m "Se argega el proyecto"

[main 77e237e] Se argega el proyecto
 1 file changed, 85 insertions(+), 64 deletions(-)


In [104]:
%%bash
#Subiremos esto a nuestra rama principal
git branch -M main

In [116]:
%%bash
#Cargaremos al proeycto ML_training_01.git
git remote rm origin
git remote add origin https://github.com/JAPJ182/ML_training_01.git
#git remote add origin https://github.com/<JAPJ182>:<*Neftigaza2021*>/ML_training_01.git


In [118]:
%%bash
git config --global user.email japj182@hotmail.com
git config --global user.name  "JAPJ182"
git config -l


user.email=japj182@hotmail.com
user.name=JAPJ182
core.repositoryformatversion=0
core.filemode=true
core.bare=false
core.logallrefupdates=true
remote.origin.url=https://github.com/JAPJ182/ML_training_01.git
remote.origin.fetch=+refs/heads/*:refs/remotes/origin/*


In [120]:
!git push -u origin main


fatal: could not read Username for 'https://github.com': No such device or address


CalledProcessError: Command 'b'git push -u origin main\n'' returned non-zero exit status 128.

In [111]:
%%bash
git config --global user.email japj182@hotmail.com
git config --global user.name "JAPJ182"
git push -u origin main

fatal: could not read Username for 'https://github.com': No such device or address


CalledProcessError: Command 'b'git config --global user.email japj182@hotmail.com\ngit config --global user.name "JAPJ182"\ngit push -u origin main\n'' returned non-zero exit status 128.

In [ ]:
# Enviar mi git local al remoto 
